In [ ]:
import numpy as np
import pandas as pd
import random
import cv2

In [ ]:
def polygon_overlay(img):
    h, w, _ = img.shape
    for _ in range(random.randint(1, 2)):
        pts = np.random.randint(0, min(h, w) // 3, (random.randint(3, 6), 2))
        cv2.fillPoly(img, [pts], color=tuple(np.random.randint(0, 256, 3).tolist()))
    return img

def motion_blur(img):
    ksize = random.choice([5, 9, 15])
    kernel = np.zeros((ksize, ksize))
    if random.choice(['horizontal', 'vertical']) == 'horizontal':
        kernel[ksize//2, :] = np.ones(ksize)
    else:
        kernel[:, ksize//2] = np.ones(ksize)
    return cv2.filter2D(img, -1, kernel / ksize)

def jpeg_compression(img):
    quality = random.randint(10, 70)
    _, enc = cv2.imencode('.jpg', img, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
    return cv2.imdecode(enc, 1)

# Corruption functions
def apply_corruption(image):
    funcs = [jpeg_compression, polygon_overlay, motion_blur]
    corrupted = image.copy()
    r = random.random()

    if r < 0.3:  # all
        for f in funcs:
            corrupted = f(corrupted)
    elif r < 0.6:  # any 2
        for f in random.sample(funcs, 2):
            corrupted = f(corrupted)
    elif r < 0.9:  # any 1
        corrupted = random.choice(funcs)(corrupted)
    return corrupted

# Image preprocessing function
def preprocess_image(path, lr_height=64, lr_width=64, hr_height=128, hr_width=128):
    img = cv2.imread(path.decode())
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))  # Intermediate resize if needed

    clean = img.copy()
    corrupted = apply_corruption(img.copy())

    # Normalize to [-1, 1]
    clean = (clean.astype(np.float32) / 127.5) - 1.0
    corrupted = (corrupted.astype(np.float32) / 127.5) - 1.0

    # Resize HR and LR
    hr = cv2.resize(clean, (hr_width, hr_height))
    lr = cv2.resize(corrupted, (lr_width, lr_height))

    return lr, hr

# TF wrapper
def tf_wrapper(path, lr_shape=(64, 64, 3), hr_shape=(128, 128, 3)):
    lr_height, lr_width, _ = lr_shape
    hr_height, hr_width, _ = hr_shape

    lr, hr = tf.numpy_function(
        func=preprocess_image,
        inp=[path, lr_height, lr_width, hr_height, hr_width],
        Tout=[tf.float32, tf.float32]
    )
    lr.set_shape(lr_shape)
    hr.set_shape(hr_shape)

    return lr, hr